In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta, time 
import json

with open("./db_config.json", "r") as f:
    config = json.load(f)
    
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(engine_url)
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit()

# --- 데이터베이스에서 데이터 로드하는 함수 ---
def load_table_to_df(table_name, engine):
    """지정된 테이블에서 모든 데이터를 Pandas DataFrame으로 로드합니다."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"'{table_name}' 테이블 로드 완료. {len(df)} 행.")
        # 날짜/시간 컬럼 타입 변환 (필요시)
        if 'game_date' in df.columns:
            df['game_date'] = pd.to_datetime(df['game_date'])
        if 'accident_date' in df.columns:
            df['accident_date'] = pd.to_datetime(df['accident_date'])
        if 'weather_date' in df.columns:
            df['weather_date'] = pd.to_datetime(df['weather_date'])
        # start_time, end_time, weather_time은 문자열로 로드될 수 있으므로, 필요시 time 객체로 변환
        return df
    except Exception as e:
        print(f"'{table_name}' 테이블 로드 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 발생 시 빈 DataFrame 반환

# --- 데이터 로드 ---
stadium_df = load_table_to_df('stadium', engine)
sports_game_df = load_table_to_df('sports_game', engine)
traffic_accident_df = load_table_to_df('traffic_accident', engine)
weather_df = load_table_to_df('weather', engine)

# 데이터베이스 연결 종료
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")

MySQL 데이터베이스에 성공적으로 연결되었습니다.
'stadium' 테이블 로드 완료. 60 행.
'sports_game' 테이블 로드 완료. 3631 행.
'traffic_accident' 테이블 로드 완료. 34032 행.
'weather' 테이블 로드 완료. 450811 행.

데이터베이스 연결이 종료되었습니다.


In [20]:
# 야구 관중수 정보
df_kbo_audience = pd.read_csv('./dataset/야구_kbo_관중수_2023_2024.csv', encoding='utf-8')
stadium_code_map = {
    "사직": "BS02",
    "청주": "CJ01",
    "창원": "CW01",
    "대구": "DG03",
    "대전": "DJ02",
    "광주": "GJ02",
    "문학": "IC02",
    "이천": "IH01",
    "포항": "PH02",
    "고척": "SO04",
    "잠실": "SO05",
    "수원": "SU03",
    "울산": "US03"
}

df_kbo_audience["stadium_code"] = df_kbo_audience["구장"].map(stadium_code_map)
df_kbo_audience['날짜'] = pd.to_datetime(df_kbo_audience['날짜'], format="%Y.%m.%d").dt.date
df_kbo_audience['관중수'] = df_kbo_audience['관중수'].str.replace(',', '').astype(int)
df_kbo_audience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   연도            1440 non-null   int64 
 1   날짜            1440 non-null   object
 2   요일            1440 non-null   object
 3   홈             1440 non-null   object
 4   방문            1440 non-null   object
 5   구장            1440 non-null   object
 6   관중수           1440 non-null   int32 
 7   stadium_code  1440 non-null   object
dtypes: int32(1), int64(1), object(6)
memory usage: 84.5+ KB


In [ ]:
# 축구 관중수
df_klg_audience = pd.read_csv('./dataset/축구_k-league_관중수_2023_2024.csv', encoding='utf-8')
klg_stadium_codemap = {
    "부산 아시아드": "BS01",
    "천안 종합": "CA01",
    "춘천 송암": "CC01",
    "대구iM뱅크PARK": "DG01",
    "대구 스타디움": "DG02",
    "대전 월드컵": "DJ01",
    "김천 종합": "GC01",
    "광주 전용": "GJ01",
    "강릉 종합": "GN01",
    "김포 솔터": "GP01",
    "인천 전용": "IC01",
    "제주 월드컵": "JE01",
    "전주 월드컵": "JJ01",
    "포항 스틸야드": "PH01",
    "서울 월드컵": "SO01",
    "상암 보조": "SO02",
    "목동 종합": "SO03",
    "수원 월드컵": "SU01",
    "수원 종합": "SU02",
    "울산 문수": "US01",
    "울산 종합": "US02"
}



df_klg_audience["stadium_code"] = df_klg_audience["경기장"].map(klg_stadium_codemap)
df_klg_audience['일자'] = pd.to_datetime(df_klg_audience['일자'], format="%Y.%m.%d").dt.date
df_klg_audience['관중수'] = df_klg_audience['관중수'].str.replace(',', '').astype(int)
df_klg_audience['원정팀'] = df_klg_audience['원정팀'].str.replace(',', '').astype(int)
df_klg_audience.head()

,시즌,대회,대회명,경기번호,일자,홈팀,상대,경기결과,경기장,관중수,원정팀,비고,stadium_code
0,2023,K리그1,하나원큐 K리그1 2023,1,2023-02-25,울산,전북,2:01,울산 문수,28039,2037,주말,US01
1,2023,K리그1,하나원큐 K리그1 2023,2,2023-02-25,서울,인천,2:01,서울 월드컵,22204,3216,주말,SO01
2,2023,K리그1,하나원큐 K리그1 2023,3,2023-02-25,수원,광주,0:01,수원 월드컵,10348,409,주말,SU01
3,2023,K리그1,하나원큐 K리그1 2023,4,2023-02-26,포항,대구,3:02,포항 스틸야드,14089,1152,주말,PH01
4,2023,K리그1,하나원큐 K리그1 2023,5,2023-02-26,제주,수원FC,0:00,제주 월드컵,8362,181,주말,JE01


In [ ]:
# 배구 관중수
df_vlg_audience = pd.read_csv('./dataset/배구_vl_관중수_2023_2024.csv', encoding='utf-8')
vlg_stadium_codemap = {
    "안산상록수체육관": "AS01",
    "천안유관순체육관": "CA02",
    "대전충무체육관": "DJ04",
    "김천실내체육관": "GC02",
    "구미박정희체육관": "GM01",
    "페퍼스타디움": "HS01",
    "화성종합실내체육관": "HS02",
    "인천계양체육관": "IC03",
    "인천삼산월드체육관": "IC04",
    "서울장충체육관": "SO08",
    "수원체육관": "SU05",
    "의정부체육관": "UJ01",
    "경민대학교 기념관(체육관)": "UJ02"
}

df_vlg_audience["stadium_code"] = df_vlg_audience["place"].map(vlg_stadium_codemap)
df_vlg_audience["gDate"] = pd.to_datetime(df_vlg_audience["gDate"], format="%Y.%m.%d").dt.date
#print(df_vlg_audience['gDate'].apply(type).value_counts())
df_vlg_audience.head()

,hTeamCode,aTeamCode,gDate,sTime,eTime,hSetPoint,aSetPoint,place,city,spectators,aTeamName,totalTime,fetch_gNum,fetch_gPart_success,stadium_code
0,1001,1004,2022-10-22,14:00,16:03,3,1,인천계양체육관,인천,1987,KB손해보험,1:52,1,201,IC03
1,2001,2002,2022-10-22,14:00,15:29,3,0,수원체육관,수원,2913,한국도로공사,1:21,2,201,SU05
2,1008,1006,2022-10-23,14:25,15:57,0,3,안산상록수체육관,안산,1661,한국전력,1:24,3,201,AS01
3,2006,2005,2022-10-23,16:00,17:31,0,3,화성종합실내체육관,화성,2579,GS칼텍스,1:23,4,201,HS02
4,1002,1005,2022-10-25,19:00,21:05,1,3,대전충무체육관,대전,931,현대캐피탈,1:54,5,201,DJ04


In [ ]:
# 야구 관중수 병합
sports_game_with_audience_df = sports_game_df.copy()
sports_game_with_audience_df['audience'] = pd.NA

sports_game_with_audience_df.loc[sports_game_with_audience_df['stadium_code'] == 'DJ03', 'stadium_code'] = 'DJ02'

df_kbo_audience_prepared = df_kbo_audience.copy()
df_kbo_audience_prepared['date_kbo'] = pd.to_datetime(df_kbo_audience_prepared['날짜'])

# 관중수 컬럼명을 임시로 변경 (충돌 방지)
df_kbo_audience_prepared.rename(columns={'관중수':'audience_temp_kbo'}, inplace=True)
# KBO 데이터 병합
sports_game_with_audience_df = sports_game_with_audience_df.merge(
    df_kbo_audience_prepared[['stadium_code', 'date_kbo', 'audience_temp_kbo']],
    left_on=['stadium_code', 'game_date'],
    right_on=['stadium_code', 'date_kbo'],
    how='left'
)

# 'audience' 컬럼 업데이트 및 임시 컬럼 삭제
sports_game_with_audience_df['audience'] = sports_game_with_audience_df['audience'].combine_first(sports_game_with_audience_df['audience_temp_kbo'])
sports_game_with_audience_df.drop(columns=['date_kbo', 'audience_temp_kbo'], inplace=True, errors='ignore')

print("--- KBO 병합 후 ---")
print(sports_game_with_audience_df)
sports_game_with_audience_df.info()
print("\n")


--- KBO 병합 후 ---
      game_id stadium_code sports_type  game_date day_of_week is_holiday  \
0           1         DJ02          야구 2023-03-13           월       None   
1           2         SO04          야구 2023-03-13           월       None   
2           3         CW01          야구 2023-03-13           월       None   
3           4         BS02          야구 2023-03-13           월       None   
4           5         DG03          야구 2023-03-13           월       None   
...       ...          ...         ...        ...         ...        ...   
3672     3627         HS01        여자배구 2024-12-26           목       None   
3673     3628         HS02        여자배구 2024-12-27           금       None   
3674     3629         IC04        여자배구 2024-12-28           토       None   
3675     3630         HS01        여자배구 2024-12-29           일       None   
3676     3631         DJ04        여자배구 2024-12-31           화       None   

          start_time        end_time  home_team_win match_type  audien

C:\Users\stagv\AppData\Local\Temp\ipykernel_33012\2376585274.py:20: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  sports_game_with_audience_df['audience'] = sports_game_with_audience_df['audience'].combine_first(sports_game_with_audience_df['audience_temp_kbo'])


In [ ]:
# 축구 관중수 병합
df_klg_audience_prepared = df_klg_audience.copy()
df_klg_audience_prepared['date_klg'] = pd.to_datetime(df_klg_audience_prepared['일자'])

# 관중수 컬럼명을 임시로 변경 (충돌 방지)
df_klg_audience_prepared.rename(columns={'관중수':'audience_temp_klg'}, inplace=True)
# KBO 데이터 병합
sports_game_with_audience_df = sports_game_with_audience_df.merge(
    df_klg_audience_prepared[['stadium_code', 'date_klg', 'audience_temp_klg']],
    left_on=['stadium_code', 'game_date'],
    right_on=['stadium_code', 'date_klg'],
    how='left'
)

# 'audience' 컬럼 업데이트 및 임시 컬럼 삭제
sports_game_with_audience_df['audience'] = sports_game_with_audience_df['audience'].combine_first(sports_game_with_audience_df['audience_temp_klg'])
sports_game_with_audience_df.drop(columns=['date_klg', 'audience_temp_klg'], inplace=True, errors='ignore')

print("--- KLG 병합 후 ---")
print(sports_game_with_audience_df)
sports_game_with_audience_df.info()
print("\n")

--- KLG 병합 후 ---
      game_id stadium_code sports_type  game_date day_of_week is_holiday  \
0           1         DJ02          야구 2023-03-13           월       None   
1           2         SO04          야구 2023-03-13           월       None   
2           3         CW01          야구 2023-03-13           월       None   
3           4         BS02          야구 2023-03-13           월       None   
4           5         DG03          야구 2023-03-13           월       None   
...       ...          ...         ...        ...         ...        ...   
3672     3627         HS01        여자배구 2024-12-26           목       None   
3673     3628         HS02        여자배구 2024-12-27           금       None   
3674     3629         IC04        여자배구 2024-12-28           토       None   
3675     3630         HS01        여자배구 2024-12-29           일       None   
3676     3631         DJ04        여자배구 2024-12-31           화       None   

          start_time        end_time  home_team_win match_type  audien

In [25]:
# 배구 관중수 병합
df_vlg_audience_prepared = df_vlg_audience.copy()
df_vlg_audience_prepared['date_vlg'] = pd.to_datetime(df_vlg_audience_prepared['gDate'])

# 관중수 컬럼명을 임시로 변경 (충돌 방지)
df_vlg_audience_prepared.rename(columns={'spectators':'audience_temp_vlg'}, inplace=True)
# KBO 데이터 병합
sports_game_with_audience_df = sports_game_with_audience_df.merge(
    df_vlg_audience_prepared[['stadium_code', 'date_vlg', 'audience_temp_vlg']],
    left_on=['stadium_code', 'game_date'],
    right_on=['stadium_code', 'date_vlg'],
    how='left'
)

# 'audience' 컬럼 업데이트 및 임시 컬럼 삭제
sports_game_with_audience_df['audience'] = sports_game_with_audience_df['audience'].combine_first(sports_game_with_audience_df['audience_temp_vlg'])
sports_game_with_audience_df.drop(columns=['date_vlg', 'audience_temp_vlg'], inplace=True, errors='ignore')

print("--- VLG 병합 후 ---")
print(sports_game_with_audience_df)
sports_game_with_audience_df.info()
print("\n")

--- VLG 병합 후 ---
      game_id stadium_code sports_type  game_date day_of_week is_holiday  \
0           1         DJ02          야구 2023-03-13           월       None   
1           2         SO04          야구 2023-03-13           월       None   
2           3         CW01          야구 2023-03-13           월       None   
3           4         BS02          야구 2023-03-13           월       None   
4           5         DG03          야구 2023-03-13           월       None   
...       ...          ...         ...        ...         ...        ...   
3672     3627         HS01        여자배구 2024-12-26           목       None   
3673     3628         HS02        여자배구 2024-12-27           금       None   
3674     3629         IC04        여자배구 2024-12-28           토       None   
3675     3630         HS01        여자배구 2024-12-29           일       None   
3676     3631         DJ04        여자배구 2024-12-31           화       None   

          start_time        end_time  home_team_win match_type  audien

### 농구 관중수도 병합해보려 했으나 보유한 데이터와 실제 데이터의 갯수가 맞지않아 경기기록과 일치가 불가하여 드랍

In [26]:
# 파일로 출력
sports_game_with_audience_df.to_csv('./dataset/sports_game_with_audience.csv')

In [28]:
# 데이터 베이스에 삽입

from sqlalchemy import create_engine, text, Table, MetaData, update
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np

with open("./db_config.json", "r") as f:
    config = json.load(f)
    
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(engine_url)
metadata = MetaData()

# sports_game 테이블 객체 로드 (테이블이 DB에 이미 존재해야 함)
# autoload_with를 사용하면 DB에서 테이블 구조를 읽어옴
try:
    sports_game_table = Table('sports_game', metadata, autoload_with=engine)
except SQLAlchemyError as e:
    print(f"Error loading table 'sports_game': {e}")
    print("Please ensure the table exists in the database and connection details are correct.")
    exit()

# --- 데이터 업데이트 ---
updated_rows_count = 0
skipped_due_to_nan_audience = 0 # audience가 NaN/NA인 경우 업데이트를 건너뛸지 여부
sports_game_with_audience_df = pd.read_csv('./dataset/sports_game_with_audience.csv')

# 트랜잭션 시작
with engine.connect() as connection:
    with connection.begin(): # 트랜잭션 시작
        for index, row in sports_game_with_audience_df.iterrows():
            stadium_code_val = row['stadium_code']
            sports_type_val = row['sports_type']
            game_date_val = row['game_date'] # Pandas Timestamp 객체
            audience_val = row['audience']

            # audience 값이 pd.NA 또는 np.nan 인 경우 SQL NULL로 처리됨
            # 만약 audience 값이 유효한 숫자일 때만 업데이트하고 싶다면, 아래 주석을 해제
            if pd.isna(audience_val):
                skipped_due_to_nan_audience += 1
                print(f"Skipping row {index} due to NaN audience: {stadium_code_val}, {sports_type_val}, {game_date_val}")
                continue
            
            # audience_val이 pd.NA면 SQLAlchemy가 알아서 NULL로 변환해줌
            # 만약 DB의 audience 컬럼이 INT이고, DataFrame의 audience가 float (np.nan 포함)이라면,
            # 유효한 숫자는 정수로 변환해주는 것이 좋음. pd.NA는 None으로 취급.
            if pd.notna(audience_val) and isinstance(audience_val, (float, np.floating)): # np.nan은 float임
                 audience_to_set = int(audience_val)
            elif pd.isna(audience_val): # pd.NA 또는 np.nan
                 audience_to_set = None # SQL NULL
            else: # 정수형 pd.NA가 아닌 경우 (이미 Int64로 변환되었거나 문자열 등)
                 audience_to_set = audience_val # int(audience_val) 하거나 그대로 둠

            try:
                stmt = (
                    update(sports_game_table)
                    .where(sports_game_table.c.stadium_code == stadium_code_val)
                    .where(sports_game_table.c.sports_type == sports_type_val)
                    .where(sports_game_table.c.game_date == game_date_val) # game_date가 DB에서 DATE 타입이면 .date() 필요할 수 있음
                    .values(audience=audience_to_set)
                )
                result = connection.execute(stmt)
                if result.rowcount > 0:
                    updated_rows_count += result.rowcount
                # else:
                #     print(f"No matching row found in DB for: {stadium_code_val}, {sports_type_val}, {game_date_val.date()}")

            except SQLAlchemyError as e:
                print(f"Error updating row {index} ({stadium_code_val}, {sports_type_val}, {game_date_val}): {e}")
            except Exception as ex: # Pandas Timestamp 관련 오류 등
                print(f"General error processing row {index} ({stadium_code_val}, {sports_type_val}, {game_date_val}): {ex}")


    # 트랜잭션이 성공적으로 끝나면 여기서 자동으로 commit됨
    # 오류 발생 시 자동으로 rollback됨

print(f"--- 업데이트 완료 ---")
print(f"총 {updated_rows_count} 개의 행의 audience 정보가 DB에 반영(또는 시도)되었습니다.")
if skipped_due_to_nan_audience > 0:
    print(f"{skipped_due_to_nan_audience} 개의 행은 audience 값이 NaN/NA 여서 건너뛰었습니다.")

Skipping row 0 due to NaN audience: DJ02, 야구, 2023-03-13
Skipping row 1 due to NaN audience: SO04, 야구, 2023-03-13
Skipping row 2 due to NaN audience: CW01, 야구, 2023-03-13
Skipping row 3 due to NaN audience: BS02, 야구, 2023-03-13
Skipping row 4 due to NaN audience: DG03, 야구, 2023-03-13
Skipping row 5 due to NaN audience: DJ02, 야구, 2023-03-14
Skipping row 6 due to NaN audience: SO04, 야구, 2023-03-14
Skipping row 7 due to NaN audience: CW01, 야구, 2023-03-14
Skipping row 8 due to NaN audience: BS02, 야구, 2023-03-14
Skipping row 9 due to NaN audience: DG03, 야구, 2023-03-14
Skipping row 10 due to NaN audience: SO04, 야구, 2023-03-15
Skipping row 11 due to NaN audience: DJ02, 야구, 2023-03-15
Skipping row 12 due to NaN audience: DG03, 야구, 2023-03-15
Skipping row 13 due to NaN audience: CW01, 야구, 2023-03-15
Skipping row 14 due to NaN audience: BS02, 야구, 2023-03-15
Skipping row 15 due to NaN audience: SO04, 야구, 2023-03-16
Skipping row 16 due to NaN audience: DJ02, 야구, 2023-03-16
Skipping row 17 due to N

In [29]:
# 데이터베이스 연결 종료
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")


데이터베이스 연결이 종료되었습니다.
